<a href="https://colab.research.google.com/github/Mazihell/T7Const_Interp_v1.2/blob/main/T7ConstInter_v1_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import nltk
import re
#tokenizer
nltk.download('punkt')
#tagger pré treinado em inglês
nltk.download('averaged_perceptron_tagger')
import numpy as np
#palavras para exclusão
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [105]:
#Artigos em ingles para donwload
link1 = Request('https://link.springer.com/article/10.1007/s42979-021-00592-x',headers={'User-Agent': 'Mozilla/5.0'})
link2 = Request('https://link.springer.com/article/10.1007/s42979-022-01371-y',headers={'User-Agent': 'Mozilla/5.0'})
link3 = Request('https://www.ibm.com/cloud/learn/neural-networks',headers={'User-Agent': 'Mozilla/5.0'})
link4 = Request('https://realpython.com/python-ai-neural-network/',headers={'User-Agent': 'Mozilla/5.0'})
link5 = Request('https://monkeylearn.com/text-classification/',headers={'User-Agent': 'Mozilla/5.0'})

In [106]:
#armazenando da pagina na variavel
pagina1 = urlopen(link1).read().decode('utf-8', 'ignore')
pagina2 = urlopen(link2).read().decode('utf-8', 'ignore')
pagina3 = urlopen(link3).read().decode('utf-8', 'ignore')
pagina4 = urlopen(link4).read().decode('utf-8', 'ignore')
pagina5 = urlopen(link5).read().decode('utf-8', 'ignore')

In [107]:
soup1 = BeautifulSoup(pagina1, "lxml")
soup2 = BeautifulSoup(pagina2, "lxml")
soup3 = BeautifulSoup(pagina3, "lxml")
soup4 = BeautifulSoup(pagina4, "lxml")
soup5 = BeautifulSoup(pagina5, "lxml")
#busca pelo ID do que precisa do html
texto1 = soup1.find(id="Sec5-content").text
texto2 = soup2.find(id="Sec2-content").text
texto3 = soup3.find(id="layout-section-second").text
texto4 = soup4.find(id="artificial-intelligence-overview").text
texto5 = soup5.find(id="___gatsby").text  
#soup3.find_all(id=True)

In [108]:
dados = [texto1, texto2, texto3, texto4, texto5]

In [109]:
# criando os objetos com as expressões regulares
remove_espec_carac = re.compile('[/(){}\[\]\|@,;]')
remove_symb = re.compile('[^0-9a-z #+_]')

# criando um objeto para remoção de stopwords no idioma ingles
stopwords = stopwords.words('english')

# criando a função
def text_prepare(text):
    
    # normalizando texto em letras minúsculas
    text = text.lower()
    
    # substituindo caracteres especiais por espaços em branco
    text = remove_espec_carac.sub(' ', text)
    
    # retornando apenas letras e números
    text = remove_symb.sub('', text)
    
    # removendo as stopwords
    text = ' '.join(word for word in text.split() if word not in stopwords)
    
    # retornando o texto modificado
    return text

In [110]:
# aplicando a função no texto (pre processamento do texto)
texto = [text_prepare(x) for x in dados]

In [111]:
#histograma (quantidade de vezes que determinada palavra aparece no texto)
word2count = {}
for data in texto:
    words = nltk.word_tokenize(data)
    for word in words:
        if word not in word2count.keys():
            word2count[word] = 1
        else:
            word2count[word] += 1


In [112]:
# biblioteca para ordenação
import heapq

# ordena a lista para saber qual é a palavra que mais se repete
freq_words = heapq.nlargest(10000,word2count, key=word2count.get)
#print (freq_words)


In [113]:
#criando o bag
import numpy as np
x = []
for data in texto:
    vector = []
    for word in freq_words:
        if word in nltk.word_tokenize(data):
            vector.append(1) # se a palavra está contida no documento coloca-se 1
        else:
            vector.append(0) # se a palavra não está contida no documento coloca-se 0
    x.append(vector)
    
y = np.asarray(x)

print(y)

[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [1 1 1 ... 1 1 1]]


In [114]:
# criando o IDF
word_idfs = {}
for word in freq_words:
    doc_count = 0 
    for data in texto:
        if word in nltk.word_tokenize(data):
            doc_count += 1
    word_idfs[word] = np.log((len(texto)/doc_count)+1)

#deixei comentado o print professor, porque ficou muito grande a quantidade de item na tela.
#print (word_idfs)

In [115]:
# criando o TF
tf_matrix = {}

for word in freq_words:
    doc_tf = []
    for data in texto:
        frequency = 0
        for w in nltk.word_tokenize(data):
            if w == word:
                frequency += 1
        tf_word = frequency/len(nltk.word_tokenize(data))
        doc_tf.append(tf_word)
    tf_matrix[word] = doc_tf

#print (tf_matrix)

In [116]:
# TF-IDF cálculo
tfidf_matrix = []

for word in tf_matrix.keys():
    tfidf = []
    for value in tf_matrix[word]:
        score = value * word_idfs[word]
        tfidf.append(score)
    tfidf_matrix.append(tfidf)
    
#print (tfidf_matrix)

In [117]:
vec = TfidfVectorizer()
matrix= vec.fit_transform(texto)
print("Nome das variáveis n",vec.get_feature_names_out(),'\n')
print("Matrix n",matrix.shape,"n",matrix.toarray())


Nome das variáveis n ['000' '02' '048' ... 'zeror' 'zerovariance' 'zip'] 

Matrix n (5, 2926) n [[0.         0.         0.         ... 0.0064832  0.0064832  0.        ]
 [0.         0.         0.02289695 ... 0.         0.         0.        ]
 [0.         0.02042147 0.         ... 0.         0.         0.02042147]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.01291697 0.         0.         ... 0.         0.         0.        ]]


In [118]:
#sklearn cossine similarity
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import sklearn as sk
df = pd.DataFrame(matrix.toarray(), columns=vec.get_feature_names())
df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,000,02,048,081,0once,10,100,101,102,104,...,youre,yourself1,youve,zaki,zaxis,zendesk,zero,zeror,zerovariance,zip
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.005231,0.006483,0.000000,0.012966,0.012966,...,0.000000,0.000000,0.000000,0.006483,0.000000,0.000000,0.01945,0.006483,0.006483,0.000000
1,0.000000,0.000000,0.022897,0.022897,0.000000,0.018473,0.000000,0.022897,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2,0.000000,0.020421,0.000000,0.000000,0.020421,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.016476,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.020421
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.070943,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
4,0.012917,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.004306,0.004306,0.000000,0.004306,0.004306,0.00000,0.000000,0.000000,0.000000


In [119]:
df2 = pd.DataFrame(cosine_similarity(df,dense_output=True))
df2.head()

,0,1,2,3,4
0,1.000000,0.157824,0.282916,0.329852,0.207403
1,0.157824,1.000000,0.079420,0.104079,0.062893
2,0.282916,0.079420,1.000000,0.255659,0.106212
3,0.329852,0.104079,0.255659,1.000000,0.157657
4,0.207403,0.062893,0.106212,0.157657,1.000000


In [120]:
t = []
for j,k in enumerate(df2.values):
  for n in range(len(k)):
    t.append([j,n,k[n]])

qq = []
for i in range(len(t)):
  if t[i][0] == t[i][1]:
    qq.append([t[i][0],t[i][1],0])
  else:
    qq.append(t[i])
qq[:5]

[[0, 0, 0],
 [0, 1, 0.1578236771958124],
 [0, 2, 0.28291575578017153],
 [0, 3, 0.3298515712309559],
 [0, 4, 0.20740295422415944]]

In [121]:
from collections import defaultdict
u=defaultdict(list)

for i in range(len(qq)):
  u[qq[i][0]].append(qq[i][2])
updated_df = pd.DataFrame(u)

position_maxVal=[]
for i in range(len(updated_df)):
  position_maxVal.append(np.argmax(updated_df[i]))

In [122]:
sent_comp = []
for j in position_maxVal:
  sent_comp.append(texto[j])
similar_texto = pd.DataFrame(sent_comp,columns=['Similar Texto'])
similarity_value = pd.DataFrame(round(updated_df.max(axis=1),4),columns=['Similarity Value'])
similarity_value


,Similarity Value
0,0.3299
1,0.1578
2,0.2829
3,0.3299
4,0.2074


In [125]:
p_texto = pd.DataFrame(texto,columns=['Parsed Texto'])
cos_sim_df = pd.concat([p_texto,similar_texto,similarity_value],axis=1)
cos_sim_df

,Parsed Texto,Similar Texto,Similarity Value
0,section discuss various machine learning algor...,artificial intelligence overviewin basic terms...,0.3299
1,interpretable ml developed explain blackbox mo...,section discuss various machine learning algor...,0.1578
2,neural networksneural networks reflect behavio...,section discuss various machine learning algor...,0.2829
3,artificial intelligence overviewin basic terms...,section discuss various machine learning algor...,0.3299
4,const tundefinedtypeof htmlimageelementloading...,section discuss various machine learning algor...,0.2074


In [127]:
print(cos_sim_df['Parsed Texto'][1])
print('----------------------------------')
print(cos_sim_df['Similar Texto'][1])
print('----------------------------------')
print('Calculo da Similaridade:',cos_sim_df['Similarity Value'][1])

interpretable ml developed explain blackbox models 10 heterogeneous etiology ad completely understood yet interpretability important enables validation biological plausibility ml models recently studies used interpretable ml ad detectionfor example long shortterm memory lstm 11 based recurrent neural networks rnn 12 trained classify cn vs mci subjects 13 experiments included multiple techniques fuse sociodemographic genetic data magnetic resonance imaging mri scans resulting models evaluated two ad datasetsthe ad subset 14 heinz nixdorf risk factors evaluation coronary calcification lifestyle recall hnr 15 61 mci 59 cn 624 subjects 397 mci 227 cn alzheimers disease neuroimaging initiative adni 16 study phase 1 visually explain individual model decisions gradientweighted class activation mapping gradcam 17 used focus biologically plausible regions observedfour heatmap visualization methodssensitivity analysis 18 guided backpropagation 19 occlusion 20 brain area occlusion inspired 21 com